In [13]:
import tensorflow as tf
import os
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, BatchNormalization, Dropout,RandomFlip, RandomRotation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
import pandas as pd
import shutil
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix
from cnn_utils import *

In [14]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
fp ='dataset_1'
filepath= f'drive/MyDrive/{fp}'
shutil.copytree(filepath,fp)


In [ ]:
!nvidia-smi

In [ ]:
METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve]

In [ ]:


def run_model(epochs,model_run,lr,batch_size,model)
    if not os.path.exists('drive/MyDrive/model_checkpoints'):
      os.mkdir('drive/MyDrive/model_checkpoints')

    outpath = os.path.join('drive/MyDrive/model_checkpoints',model_run)
    if not os.path.exists(outpath):
      os.mkdir(outpath)
    callbacks = [EarlyStopping(patience = 1,restore_best_weights=True),
                 ModelCheckpoint(outpath,save_best_only=True)]



    AUTOTUNE = tf.data.AUTOTUNE
    batch_size= 32
    training, val = load_data(fp,batch_size)
    training.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
    val.cache().prefetch(buffer_size=AUTOTUNE)
    
    opt=Adam(learning_rate=lr)
    model.compile(optimizer=opt,loss=tf.keras.losses.BinaryCrossentropy(),metrics=metrics)
    
    history = model.fit(training, validation_data=val,epochs=epochs,callbacks = callbacks)
    
    test_preds, test_labels = get_predictions(model,val)
    plot_confusion_matrix(test_preds, test_labels)
    plot_confusion_matrix(test_preds, test_labels,p=0.95)
    return history



In [ ]:
epochs = 10
model_run = '7_run1'
lr = 0.001


cnn = models.Sequential([layers.Rescaling(1./255,
                                           input_shape=(256,256,3))])
cnn.add(augment())
cnn = conv_block(cnn,64,dropout=0.2,batchnorm=True)
cnn=conv_block(cnn,128,dropout=0.2,batchnorm=True)
cnn=conv_block(cnn,256,depth=4,dropout=0.2,batchnorm=True)
cnn.add(layers.Flatten())
cnn.add(layers.Dense(256,activation='relu'))
cnn.add(layers.Dense(1,activation='sigmoid'))

cnn.summary()

run_model(epochs,model_run,lr,batchsize,cnn)